In [1]:
import pandas as pd
import xlrd
import os
from collections import defaultdict

In [2]:
#from PyPDF2 import PdfFileReader,PdfFileWriter

In [3]:
ls ./data

2019_12/            2020_04的副本/      2020_07/            2020_11/
2020_01/            2020_05/            2020_08/            季度汇总/
2020_03/            2020_05的副本/      2020_09/            和黄晴的结算/
2020_04/            2020_06/            2020_10/


In [4]:
class Bill:
    def __init__(self, year, mon):
        self.year = year
        self.mon = mon
        self.sell_file = self.get_sell_file()
        content = xlrd.open_workbook(filename=self.sell_file,encoding_override='gbk')
        self.sell_df = pd.read_excel(content)
        self.commission_file = self.get_commission_file()
        content = xlrd.open_workbook(filename=self.commission_file,encoding_override='gbk')
        self.commission_df = pd.read_excel(content)
        #self.sell_df = pd.read_excel(self.sell_file,encoding='gbk')
    def gen_path(self, filename):
        return  './data/{0}_{1:02d}/{2}'.format(self.year, self.mon,filename)
    def get_sell_file(self):
        return self.gen_path('销售单据表.xls')
    def get_commission_file(self):
        return self.gen_path('提成明细表.xls')
    def get_total_income(self):
        #总收入
        self.total_income = 0
        #充值总收入
        self.total_recharge = 0
        #充次总收入
        self.total_time_recharge = 0
        for index, row in self.sell_df.iterrows():
            income_sf = row['实付']
            income_czk = row['储值卡']
            self.total_income += row['实付'] - row['储值卡']
            
            if row['单据类型'] == '购买次数':
                self.total_time_recharge += row['实付']
                
            if row['单据类型'] == '购买次数' or row['单据类型'] == '充值收费' or row['单据类型']=='开户收费':
                self.total_recharge += row['实付'] - row['储值卡']
               
        print('总收入:',self.total_income)
        print('充值总收入:',self.total_recharge)
        print('充次总收入:',self.total_time_recharge) 
    def get_true_income(self):
        # 实际项目总收入（会员+非会员）
        self.true_income = 0
        true_income_types = ['消费收银','快速消费']
        #会员实际项目总收入
        self.member_true_income = 0
        for index, row in self.sell_df.iterrows():
            if row['单据类型'] in true_income_types:
                if not pd.isnull(row['会员类型']):
                    self.member_true_income += row['实付']
                self.true_income += row['实付']
        print(self.member_true_income)
        print(self.true_income)
        
        #实际扣次项目
        self.true_time_dic = defaultdict(int)
        for index, row in self.commission_df.iterrows():
            if row['提成类型'] == '扣次消费':
                self.true_time_dic[row['名称']] += 1
        print(self.true_time_dic)
    def get_employee_commission(self):
        tmp_commission = self.commission_df.copy()
        self.commision_detail = {}
        selected_df = tmp_commission[['时间','员工','名称','支付','提成','提成类型','姓名']]
        selected_df['时间'] = selected_df['时间'].apply(lambda x:'{0}-{1}'.format(x.month, x.day))
        for employee, df in selected_df.groupby(['员工']):
            print(employee)
            print(df)
            df = df[['时间','员工','名称','提成类型','姓名','支付','提成']]
            print(df['提成'].sum())
            self.commision_detail[employee +"_提成"] = df['提成'].sum()
            df.to_csv(self.gen_path('{0}.csv'.format(employee)),index=False,encoding='gbk')
            df.to_excel(self.gen_path('{0}.xls'.format(employee)),index=False,encoding='gbk')
        self.commision_detail['总提成'] = tmp_commission['提成'].sum()
        print(self.commision_detail)
        
    def get_recharge_detail(self):
        tmp_df = self.sell_df.copy()
        recharge_df = tmp_df[tmp_df['单据类型'].isin(set(['购买次数','充值收费','开户收费']))]
        recharge_df = recharge_df[['单据类型','时间','实付','储值卡','会员姓名','会员卡号']]
        total_recharge = recharge_df['实付'].sum()-recharge_df['储值卡'].sum()
        des_sum = '累计充值：{0},各个会员充值情况如下\n'.format(total_recharge)
        des_sum += '==============================\n'
        for name, df in recharge_df.groupby(['会员卡号']):
            s = 0
            des = '{0}({1:d}):累计充值{2}元\n'.format(df.iloc[0]['会员姓名'],int(name),int(df['实付'].sum()-df['储值卡'].sum()))
            des_sum += des
            #print('{0}:{1}'.format(name,df['实付'].sum()-df['储值卡'].sum()))
            total_recharge += df['实付'].sum()-df['储值卡'].sum()
            s += df['实付'].sum()-df['储值卡'].sum()
            des = '明细如下:\n'
            df = df.sort_values(by=['单据类型'],ascending=False)

            for index, row in df.iterrows():
                if row['单据类型'] == '购买次数' and row['实付']>0:
                    des = des + '购买次数:' + str(int((row['实付']))) + ',时间:' + (str(row['时间'].month) + '-' + str(row['时间'].day))+'\n'
                    s -= row['实付']
                if row['单据类型'] == '充值收费' and s > 0:
                    des = des + '充值收费:' + str((int(row['实付']))) + ',时间:' + (str(row['时间'].month) + '-' + str(row['时间'].day))+'\n'
                    s -= row['实付']
                if row['单据类型'] == '开户收费' and s > 0:
                    des = des + '开户收费:' + str((int(row['实付']))) + ',时间:' + (str(row['时间'].month) + '-' + str(row['时间'].day))+'\n'
                    s -= row['实付']
            #print(des)
            des_sum += des
            des_sum += '==============================\n'
            f = open(self.gen_path('会员充值明细_{0}_{1}.txt'.format(self.year,self.mon)),'w')
            f.write(des_sum)
            f.close()
        return des_sum
    def get_nomember_detail(self):
        tmp_df = self.sell_df.copy()
        tmp_df['not_member'] = tmp_df['会员卡号'].apply(lambda x:pd.isnull(x))
        nomember_detail_df = tmp_df[tmp_df['not_member']]
        nomember_detail_df = nomember_detail_df[['单据类型','实付','时间']]
        #print(nomember_detail_df)
        des_not_member = ''
        des_not_member += '散客累计消费:{0}\n明细如下：\n'.format(nomember_detail_df['实付'].sum())
        for index, row in nomember_detail_df.iterrows():
            des_not_member += '{0}:'.format(row['单据类型']) + str((int(row['实付']))) + ',时间:' + (str(row['时间'].month) + '-' + str(row['时间'].day))+'\n'
        f = open(self.gen_path('散客消费明细_{0}_{1}.txt'.format(self.year,self.mon)),'w')
        f.write(des_not_member)
        f.close()
        return nomember_detail_df['实付'].sum()

In [5]:
ls ./data/2020_08/

孙妮.csv                       提成明细表.xls
黄晴.csv                       销售单据表.xls
孙妮.xls                       提成明细表_bak.xls
黄晴.xls                       销售单据表_bak.xls
邵红梅.csv                     会员充值明细_2020_8.txt
邵红梅.xls                     散客消费明细_2020_8.txt


In [6]:
ls ./data/2020_08/提成明细表.xls

./data/2020_08/提成明细表.xls


In [7]:
bill = Bill(2020, 11)

WARNING *** file size (27796) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (30769) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


In [8]:
bill.get_sell_file()

'./data/2020_11/销售单据表.xls'

In [9]:
bill.get_true_income()

5173
6299
defaultdict(<class 'int'>, {'暖宫艾灸': 3, '皮肤清洁护理': 2, '美睫': 1})


In [10]:
bill.sell_file

'./data/2020_11/销售单据表.xls'

In [11]:
bill.get_total_income()

总收入: 8126
充值总收入: 7000
充次总收入: 0


In [12]:
bill.commission_file

'./data/2020_11/提成明细表.xls'

In [13]:
bill.get_employee_commission()

孙妮
       时间  员工    名称      支付     提成  提成类型   姓名
0    11-2  孙妮   NaN  1000.0   50.0  充值收费  城西路
1    11-2  孙妮   NaN   290.0  116.0  快速消费  城西路
2    11-2  孙妮   NaN  1000.0   50.0  充值收费    周
3    11-2  孙妮   NaN   245.0   98.0  快速消费    周
4    11-3  孙妮   NaN  1000.0   50.0  充值收费   徐梅
5    11-4  孙妮   NaN   240.0   96.0  快速消费   徐梅
9    11-6  孙妮   NaN   110.0   44.0  快速消费   潘潘
10   11-6  孙妮   NaN   119.0   47.6  快速消费  徐晔琼
11   11-9  孙妮   NaN  1000.0   50.0  充值收费   袁月
12   11-9  孙妮   NaN   194.0    8.0  快速消费  沈慧仙
13   11-9  孙妮  暖宫艾灸    88.0    8.0  扣次消费  沈慧仙
14   11-9  孙妮   NaN   100.0   40.0  快速消费  蒋丽红
15   11-9  孙妮   NaN   240.0   96.0  快速消费  NaN
16   11-9  孙妮   NaN   100.0   40.0  快速消费  慧慧姐
17   11-9  孙妮   NaN   110.0   44.0  快速消费   姣爷
20  11-12  孙妮   NaN   240.0   96.0  快速消费   黄燕
22  11-12  孙妮   NaN   100.0   40.0  快速消费  小微姐
23  11-12  孙妮   NaN   220.0   88.0  快速消费   钱萍
24  11-12  孙妮   NaN   150.0   60.0  快速消费  NaN
25  11-16  孙妮   NaN  1000.0   50.0  充值收费  于芈亿
26  11-16  孙妮   NaN   240.0   9

/Users/mtdp/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [14]:
des_sum = bill.get_recharge_detail()

In [15]:
des_sum

'累计充值：7000,各个会员充值情况如下\n==============================\n潘潘(60012):累计充值1000元\n明细如下:\n充值收费:1000,时间:11-6\n==============================\n袁月(60015):累计充值1000元\n明细如下:\n充值收费:1000,时间:11-9\n==============================\n于芈亿(60018):累计充值1000元\n明细如下:\n充值收费:1000,时间:11-16\n==============================\n王敏(60069):累计充值1000元\n明细如下:\n充值收费:1000,时间:11-23\n==============================\n城西路(60088):累计充值1000元\n明细如下:\n充值收费:1000,时间:11-2\n==============================\n周(60089):累计充值1000元\n明细如下:\n充值收费:1000,时间:11-2\n==============================\n徐梅(3469286837):累计充值1000元\n明细如下:\n充值收费:1000,时间:11-3\n==============================\n'

In [16]:
des_not_member = bill.get_nomember_detail()

In [17]:
des_not_member

1126

In [18]:
bills  = []
bills.append(Bill(2020, 9))
bills.append(Bill(2020, 10))
bills.append(Bill(2020, 11))

WARNING *** file size (22948) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (26908) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (27796) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (30769) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


In [19]:
rows = []
for bill in bills:
    row = {}
    bill.get_total_income()
    row['月份'] = bill.mon
    row['总收入'] = bill.total_income
    row['充值总收入'] = bill.total_recharge
    row['散客消费总收入'] = bill.get_nomember_detail()
    bill.get_employee_commission()
    print(bill.commision_detail)
    for k in  bill.commision_detail:
        row[k] = bill.commision_detail[k]
    rows.append(row)

总收入: 8906
充值总收入: 5500
充次总收入: 0
孙妮
      时间  员工    名称    支付     提成  提成类型   姓名
3    9-8  孙妮   NaN  1000   50.0  充值收费  小倪姐
4    9-8  孙妮   NaN   294  117.6  快速消费  小倪姐
5    9-8  孙妮    美甲    88   30.0  消费收银  NaN
6    9-8  孙妮   NaN    92   36.8  快速消费  NaN
7    9-9  孙妮   NaN   500   25.0  充值收费   小郭
8    9-9  孙妮    美睫   250  100.0  扣次消费   小郭
10   9-9  孙妮   NaN   198   39.6  快速消费   小郭
11  9-10  孙妮   NaN   149   59.6  快速消费  王丹婧
14  9-10  孙妮    美睫   250  100.0  扣次消费   小郭
16  9-10  孙妮   NaN   109   43.6  快速消费   小郭
17  9-14  孙妮   NaN    20    8.0  快速消费   刘梅
18  9-14  孙妮   NaN   240   96.0  快速消费  NaN
19  9-15  孙妮   NaN  1000   50.0  充值收费   吴雨
20  9-15  孙妮   NaN   313  125.2  快速消费   吴雨
21  9-17  孙妮   NaN   283  113.2  快速消费  王丹婧
22  9-17  孙妮   NaN    69   17.6  快速消费   王敏
24  9-18  孙妮   NaN  1000   50.0  充值收费   张依
25  9-18  孙妮   NaN   360  130.0  快速消费   张依
27  9-19  孙妮   NaN   294  117.6  快速消费  于芈亿
28  9-19  孙妮   NaN    20    8.0  快速消费  于芈亿
32  9-20  孙妮   NaN   194   77.6  快速消费  沈慧仙
34  9-24  孙妮   NaN  

/Users/mtdp/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/mtdp/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/mtdp/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

In [20]:
rows

[{'月份': 9,
  '总收入': 8906,
  '充值总收入': 5500,
  '散客消费总收入': 3406,
  '孙妮_提成': 2678.9999999999995,
  '邵红梅_提成': 738.4,
  '总提成': 3417.3999999999996},
 {'月份': 10,
  '总收入': 6030.0,
  '充值总收入': 3898.0,
  '散客消费总收入': 2132.0,
  '孙妮_提成': 2256.24,
  '邵红梅_提成': 309.2,
  '总提成': 2565.44},
 {'月份': 11,
  '总收入': 8126,
  '充值总收入': 7000,
  '散客消费总收入': 1126,
  '孙妮_提成': 2628.0,
  '邵红梅_提成': 389.59999999999997,
  '总提成': 3017.6000000000004}]

In [21]:
result = pd.DataFrame(rows)

In [22]:
result.to_excel('季度收入汇总表_09_11.xlsx',index=False)